In [ ]:
!bash setup_videoknet_v2.sh

In [9]:
### Preparing folders

import os

folders = ['weights', 'data/inference_folder/video_sequence/val', 'model_inference_outputs', "frames", "video"]

for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)


In [32]:
!pip install gdown
!pip install opencv-python


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [37]:
import gdown
url = "https://doc-0k-4g-docs.googleusercontent.com/docs/securesc/ldlje6hgjv8k8864h8t7qvko6knd8rpk/bd7cjq26q4vegtil15bjob7m0chj8vsb/1685115450000/05650299754312687985/05650299754312687985/1KKEjRcWoWyt_RC7bXAHiKqccABGIrO0V?e=download&ax=ADWCPKA4SsN-85mZ1vNh2OWF5O6GmEpufkjEmJhL5pyoeeC54MhWIH4jK70-Cg_BZR8tuoHR27tNhPyEjeuw3obbzqOvxtlQGAjv-r17iCMPlWM0-zqed2K7wSGbMh3lnnf3lDOQnST9Cysgz0BtLVIR5Cpjtp2r07fDymvWCloBe6SrQZgekCghFeXgybReJgFmuOc8QtMxVJJvtO207sUE9NvCV9h9vlWkbe7UM2iLxDmJIZqZPW2m3OimoZwlIYl7dNlrqKg8p__uld-mkFvAWXnREvJXlca59bkDeu9Xdu8_5LgMkv7tlTQm5hOOmAACX0YDuADo68paV_rwc4nYaFwn0JnIB9peQgxx_-_HAIoyRhdLrCcLVJYr3A08VnXewPzfsCa_RJw20jdJBgeAo1XRRQV9JfdUGZqsq07sxez1M1jaru3TT-rKptA5VQ17dQUTXGB95tnHETTKINdzqFcB75P3ASkBU3llK28I5WKRoP5tpGFXH-qWrHnk4cOro4Znddcupjza44PnMIrootBqUpzjbCpauFnGZkjpNa4nz8wD4uE0Jq6gw8ohLj-s0K8VIsoaVpR1ksJXAWB1tMEGPMxnmRIMXWy5OSCxRlni12HOGSAMQBLLdOzYgCevV3Quzp3p-ksTR7QaKew5HvkTcbQjOff5ql8APthjlIyj-5FT8VTJMN9HywSK3LXAWsHDfLkOCpyKInO3EAYW_tcc1KoZXaq7qXtRbGjjKJgxf4ioDEMRYipouwz1Ad1abqgSHQixD1MPMEt2Vldn_Dtj6zeIN2rg4LFDax0zpylbgLxo31fKhZLsPW5HDvKTAbpy8SqTDdCD_mSO6-iCajqzPBPSiCHKduxSk4VJJsI1_BBkexhyN8LbUoRz7YzjCJr31UfUH_gH7Soig2n50y8OiA2DXU-SPctzKCTXiqrS-lUw1ddG2Jtw9YZmwTWiTdtBH68W7KNucjRO1694IEZGHmc2BJDKudrBq5szCzRd&uuid=3a316035-d70c-4c4d-a45c-709d6270c0d3&authuser=2&nonce=cv16bej9vgk10&user=05650299754312687985&hash=77i5br2ki41b0n584v4riqjjsqc9aeki"
out = "./weights/knet_city_step_pan_r50.pth"
url = "https://drive.google.com/file/d/1KKEjRcWoWyt_RC7bXAHiKqccABGIrO0V/view?usp=share_link"
gdown.download(url, out, quiet=False, fuzzy=True)

Downloading...
From (uriginal): https://drive.google.com/uc?id=1KKEjRcWoWyt_RC7bXAHiKqccABGIrO0V
From (redirected): https://drive.google.com/uc?id=1KKEjRcWoWyt_RC7bXAHiKqccABGIrO0V&confirm=t&uuid=9bac9c4d-611b-4baa-a544-633048bd051b
To: /workspace/minini_clean/Video-K-Net/weights/knet_city_step_pan_r50.pth
100%|██████████| 448M/448M [00:19<00:00, 22.7MB/s] 


'./weights/knet_city_step_pan_r50.pth'

### Extracting frames from folder

In [10]:
!rm ./frames/*

rm: cannot remove './frames/*': No such file or directory


In [15]:
import sys
import cv2
import os
import numpy as np
import os



VIDEO_PATH = "video/video.mp4"
RESULT_PATH = "frames"
FRAME_RATE = 20
SECONDS_TO_EXTRACT = 30 #if you want all the video put np.inf
MODEL_OUTPUT_PATH = "./model_inference_outputs"
print(f"Video path: {os.path.join(os.getcwd(), VIDEO_PATH)}")
print(f"Frames path: {os.path.join(os.getcwd(), RESULT_PATH)}")
print(f"Frame rate: {FRAME_RATE}")
print(f"Seconds to extract: {SECONDS_TO_EXTRACT}")
print("Current working directory is", os.getcwd())

if not os.path.exists(VIDEO_PATH):
    raise ValueError(f"Video path {VIDEO_PATH} does not exist")
absolute_counter = 0
# Open the video file
cap = cv2.VideoCapture(VIDEO_PATH)

# Get the frame rate of the video
fps = cap.get(cv2.CAP_PROP_FPS)

# Set the frame rate for extraction
frame_rate = FRAME_RATE

# Calculate the interval between frames
interval = round(fps / frame_rate)

# Initialize a counter for the extracted frames
count = 0

# Initialize a variable for the elapsed time
elapsed_time = 0

# Loop through the video frames

while cap.isOpened() and elapsed_time < SECONDS_TO_EXTRACT:

    
    # Read a frame from the video
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        break

    # Check if the frame should be extracted
    if count % interval == 0:
        # Save the frame as an image file
        cv2.imwrite(f'{RESULT_PATH}/000002_{absolute_counter:06d}_leftImg8bit.png', frame)
        absolute_counter += 1

    # Increment the frame counter and elapsed time
    count += 1
    elapsed_time = count / fps

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()

print("working...")
print("Extraction done!")

Video path: /workspace/minini_clean/Video-K-Net/video/video.mp4
Frames path: /workspace/minini_clean/Video-K-Net/frames
Frame rate: 20
Seconds to extract: 30
Current working directory is /workspace/minini_clean/Video-K-Net
Extraction done!


### Running inference on extracted frames

In [16]:
!rm ./data/inference_folder/video_sequence/val/* #emptying the folder

!cp ./frames/* ./data/inference_folder/video_sequence/val #copying the frames to the folder (we are assuming you already generated them with notebook

os.environ["MODEL_OUTPUT_PATH"] = MODEL_OUTPUT_PATH

!rm -r $MODEL_OUTPUT_PATH/* #we empty the prediction folder

!bash ./tools/inference.sh configs/video_knet_kitti_step/video_knet_s3_r50_rpn_1x_kitti_step_sigmoid_stride2_mask_embed_link_ffn_joint_train.py ./weights/knet_city_step_pan_r50.pth ./risultati_predictions prova_da_sh  #running the inference

rm: cannot remove './risultati_predictions/*': No such file or directory
Namespace(cfg_options=None, checkpoint='./weights/knet_city_step_pan_r50.pth', config='configs/video_knet_kitti_step/video_knet_s3_r50_rpn_1x_kitti_step_sigmoid_stride2_mask_embed_link_ffn_joint_train.py', eval=None, eval_options=None, format_only=False, fuse_conv_bn=False, gpu_collect=False, launcher='none', local_rank=0, n_video=50, options=None, out=None, output_dir='./work_dirs/vps/vps_output', pan_im_json_file='data/cityscapes_vps/panoptic_im_val_city_vps.json', semantic=False, show=False, show_dir='./risultati_predictions', show_score_thr=0.3, tmpdir=None)
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA {'type': 'KITTISTEPDVPSDataset', 'data_root': 'data/inference_folder', 'split': 'val', 'ref_seq_index': None, 'test_mode': True, 'pipeline': [{'type': 'LoadImgDirect'}, {'type': 'MultiScaleFlipAug', 'scale_factor': [1.0], 'flip': False, 'transforms': [{'type': 'RandomFlip'}, {'type': 'N

In [ ]:
### CREATING GIFS

In [18]:
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 93.9 MB/s eta 0:00:00a 0:00:01
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 104.2 MB/s eta 0:00:00
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 96.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [19]:
import hashlib
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import os
from PIL import Image
city_labels = [
    ('road', 0, (128, 64, 128)),
    ('sidewalk', 1, (244, 35, 232)),
    ('building', 2, (70, 70, 70)),
    ('wall', 3, (102, 102, 156)),
    ('fence', 4, (190, 153, 153)),
    ('pole', 5, (153, 153, 153)),
    ('traffic light', 6, (250, 170, 30)),
    ('traffic sign', 7, (220, 220, 0)),
    ('vegetation', 8, (107, 142, 35)),
    ('terrain', 9, (152, 251, 152)),
    ('sky', 10, (70, 130, 180)),
    ('person', 11, (220, 20, 60)),
    ('rider', 12, (255, 0, 0)),
    ('car', 13, (0, 0, 142)),
    ('truck', 14, (0, 0, 70)),
    ('bus', 15, (0, 60, 100)),
    ('train', 16, (0, 80, 100)),
    ('motorcycle', 17, (0, 0, 230)),
    ('bicycle', 18, (119, 11, 32)),
    ('void', 19, (0, 0, 0)),
    ('void', 255, (0, 0, 0))
]


def sha256num(num):
    hex = hashlib.sha256(str(num).encode('utf-8')).hexdigest()
    hex = hex[-6:]
    return int(hex, 16)


def id2rgb(id_map):
    if isinstance(id_map, np.ndarray):
        id_map_copy = id_map.copy()
        rgb_shape = tuple(list(id_map.shape) + [3])
        rgb_map = np.zeros(rgb_shape, dtype=np.uint8)
        for i in range(3):
            rgb_map[..., i] = id_map_copy % 256
            id_map_copy //= 256
        return rgb_map
    color = []
    for _ in range(3):
        color.append(id_map % 256)
        id_map //= 256
    return color


def cityscapes_cat2rgb(cat_map):
    color_map = np.zeros_like(cat_map).astype(np.uint8)
    color_map = color_map[..., None].repeat(3, axis=-1)
    for each_class in city_labels:
        index = cat_map == each_class[1]
        if index.any():
            color_map[index] = each_class[2]
    return color_map


def trackmap2rgb(track_map):
    color_map = np.zeros_like(track_map).astype(np.uint8)
    color_map = color_map[..., None].repeat(3, axis=-1)
    for id_cur in np.unique(track_map):
        if id_cur == 0:
            continue
        color_map[track_map == id_cur] = id2rgb(sha256num(id_cur))
    return color_map


def draw_bbox_on_img(vis_img, bboxes):
    for index in range(bboxes.shape[0]):
        cv2.rectangle(vis_img, (int(bboxes[index][0]), int(bboxes[index][1])),
                      (int(bboxes[index][2]), int(bboxes[index][3])), (0, 0, 255), thickness=1)
    return vis_img


In [21]:
#### TO PRODUCE THE GIF OF THE PANOPTIC MASKS OF THE MODEL (OUTPUTS)


def resize_img(image, basewidth = 600):
    img = image
    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img = img.resize((basewidth,hsize), Image.Resampling.LANCZOS)
    return img
absolute_count = 0
images = []

path_to_model_outputs = f"{MODEL_OUTPUT_PATH}/final"
# path_to_model_outputs = "Video-K-Net/robertominini_predictions/final"
video_sequence_id = "0002"
PATH = os.path.join(path_to_model_outputs, video_sequence_id)

for file in sorted(os.listdir(PATH)):
    if not file[-4:] == ".png":
        continue
    if absolute_count > 300:
        break
    image = Image.open(f"{path_to_model_outputs}/{video_sequence_id}/{file}")
    image = resize_img(image)
    # plt.imshow(cityscapes_cat2rgb(np.array(image))[:,:,:,0])
    images.append(Image.fromarray(cityscapes_cat2rgb(np.array(image))[:,:,:,0]))
    # images.append(cityscapes_cat2rgb(np.array(image))[:,:,:,0])
    absolute_count += 1
    
durations = [0] * len(images)
images[0].save('result.gif',
               save_all=True, append_images=images[1::2], optimize=False, duration=durations, loop=0)

print("Done!")

000000.png
000001.png
000002.png
000003.png
000004.png
000005.png
000006.png
000007.png
000008.png
000009.png
000010.png
000011.png
000012.png
000013.png
000014.png
000015.png
000016.png
000017.png
000018.png
000019.png
000020.png
000021.png
000022.png
000023.png
000024.png
000025.png
000026.png
000027.png
000028.png
000029.png
000030.png
000031.png
000032.png
000033.png
000034.png
000035.png
000036.png
000037.png
000038.png
000039.png
000040.png
000041.png
000042.png
000043.png
000044.png
000045.png
000046.png
000047.png
000048.png
000049.png
000050.png
000051.png
000052.png
000053.png
000054.png
000055.png
000056.png
000057.png
000058.png
000059.png
000060.png
000061.png
000062.png
000063.png
000064.png
000065.png
000066.png
000067.png
000068.png
000069.png
000070.png
000071.png
000072.png
000073.png
000074.png
000075.png
000076.png
000077.png
000078.png
000079.png
000080.png
000081.png
000082.png
000083.png
000084.png
000085.png
000086.png
000087.png
000088.png
000089.png
000090.png

![Result gif](result.gif)